##### Austin Hancock

## MSDS 7337 - Section 401
## Homework - #8
[Data Science @ Southern Methodist University](https://datascience.smu.edu/)

### Table of Contents
* [Description](#Description)
* [Tools](#Tools)
* [Question-1](#Question-1)
* [Question-2](#Question-2)

### <a name="Description"></a>Description
For the HW-8 assignment I will be addressing the following:

    - Load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer 
    as explained in the corresponding reference. 
    - For each of the clusters created in HW-7, compute the average, median, high, and low sentiment scores for each cluster.

### <a name="Tools"></a>Tools

In [373]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import nltk; print("NLTK", nltk.__version__)
import bs4; print("bs4", bs4.__version__)
from bs4 import BeautifulSoup
import urllib
from urllib import request
import re; print("re", re.__version__)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd; print("Pandas", pd.__version__)
import sklearn; print("sklearn", sklearn.__version__)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.cluster import KMeans
from collections import Counter
from sklearn import svm
import numpy as np; print("np", np.__version__)
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Windows-10-10.0.17134-SP0
Python 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
NLTK 3.2.4
bs4 4.6.0
re 2.2.1
Pandas 0.20.3
sklearn 0.19.1
np 1.14.2


C:\Users\ahancock\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### <a name="Question-1"></a>Question-1

Below, I am collecting the reviews from HW-5 and outputting them to a list called 'reviews'.

In [37]:
moana_url = "https://www.imdb.com/title/tt3521164/reviews?ref_=tt_ql_3"
frozen_url = 'https://www.imdb.com/title/tt2294629/reviews?ref_=tt_ql_3'
coco_url = 'https://www.imdb.com/title/tt2380307/reviews?ref_=tt_ql_3'
LionKing_url = 'https://www.imdb.com/title/tt0110357/reviews?ref_=tt_ql_3'

def find_permalinks(url):
    page = request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    review_containers = soup.find_all('div', class_ = 'review-container')
    for review in review_containers:
        permalinks.append("https://www.imdb.com" + review.find('a', attrs={'href': re.compile("/review")}).get('href'))
        
# Permalinks to reviews
permalinks = []

find_permalinks(moana_url)
find_permalinks(frozen_url)
find_permalinks(coco_url)
find_permalinks(LionKing_url)

In [248]:
# Grab the main review and rating from each link and add to list
reviews = []
ratings = []
stars = []

for link in permalinks:
    page = request.urlopen(link)
    soup = BeautifulSoup(page, 'html.parser')
    review_containers = soup.find_all('div', class_ = 'review-container')
    for review in review_containers:
        reviews.append(review.find(class_ = 'text show-more__control').text)
        ratings.append(review.find(class_ = 'rating-other-user-rating'))

# There are reviews with no rating, here I am manually assigning the star value        
count = 0
for rating in ratings:
    if count in [19, 79, 81, 83]:
        stars.append('No rating')
    else:
        stars.append(rating.span.text)
    count = count + 1

In [249]:
# Confirm extraction of reviews and rating
print(reviews[0])
print(stars[0])

Moana is a return to the classic Disney formula, the clichés and characters ripped from a number of other animated films. However, the pure beauty and skill of the production rises the old story into new heights.Following from the success of Zootropolis, Moana follows a more traditional narrative we know and love; the princess who wishes for something more and is whisked on a supernatural adventure. We know this story so well yet Moana seems fresh and thrilling as if the plot was innovative. Perhaps this is due to the Polynesian setting or the morally ambiguous Maui, played perfectly by Dwayne Johnson, but most likely it is it the simple magic of Disney – the wonder for both children and adults has reached its peak with the perfection of the classic formula. For once, the clichés make the film more enjoyable. The quality of the animation helps too: it's clear they have reached the pinnacle of blending realistic textures with stylised designs, creating an aesthetic beauty that few other

Next, I tokenize the reviews and create a tfidf matrix from a count vectorizer.

In [394]:
def remove_stopwords(tokens):
    stopword_list = nltk.corpus.stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    return filtered_tokens

In [250]:
# Tokenize
reviews_tokenized = []

for review in reviews:
    text = word_tokenize(review)
    text = [w.lower() for w in text]
    stop_words = set(stopwords.words('english'))
    filtered_sentence = []
    for w in text:
        if w not in stop_words:
            filtered_sentence.append(w)
        reviews_tokenized.append(filtered_sentence)

In [251]:
# Vectorize the reviews using CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer.fit_transform(reviews)

<100x3948 sparse matrix of type '<class 'numpy.int64'>'
	with 10917 stored elements in Compressed Sparse Row format>

In [252]:
# Create tfidf matrix
tf_matrix = count_vectorizer.transform(reviews).toarray()
tfidfTran = TfidfTransformer(norm="l2")
tfidfTran.fit(tf_matrix)
tfidf_matrix = tfidfTran.transform(tf_matrix)
print(tfidf_matrix.shape)

(100, 3948)


In [253]:
# Return feature names from the vectorizer
feature_names = count_vectorizer.get_feature_names()
print(feature_names[100:120])

['actual', 'actually', 'adam', 'adapt', 'adapted', 'adapting', 'adaption', 'add', 'added', 'adding', 'adds', 'adheres', 'adjust', 'admirable', 'admire', 'admit', 'adopt', 'adopted', 'adore', 'adored']


We now have a matrix and the features within our corpus. Using these, we can perform a K-Means clustering method.

The clustering method will cluster the reviews based on the TF-IDF features from the reviews.

In [254]:
# Define the k-means clustering function
def k_means(feature_matrix, num_clusters):
    km = KMeans(n_clusters=num_clusters, max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters

# Set number of clusters
num_clusters = 4

# Get clusters and assign the cluster labels to the reviews
km_obj, clusters = k_means(feature_matrix=tfidf_matrix, num_clusters=num_clusters)

Using this list, I next create a dataframe to see which reviews were assigned to which cluster.

In [255]:
# Create dataframe from the reviews list
reviews_df = pd.DataFrame(reviews)

# Add a new column for the movie title
reviews_df.insert(0, 'Movie', '')

# Since I pulled 25 reviews for each movie into an ordered list, I can assign the movie titles by index
reviews_df.iloc[0:25,0] = 'Moana'
reviews_df.iloc[25:50,0] = 'Frozen'
reviews_df.iloc[50:75,0] = 'Coco'
reviews_df.iloc[75:100,0] = 'LionKing'

# Rename columns to be 'Movie' and 'Review'
reviews_df.columns=['Movie', 'Review']

# Add a column for star rating
reviews_df['User_Rating'] = stars

# Add sentiment based on star rating
star_sentiment = []
for star in stars:
    if star in ['5','6','7', '8', '9', '10']:
        star_sentiment.append('Positive')
    elif star in ['0', '1', '2', '3','4']:
        star_sentiment.append('Negative')
    else:
        star_sentiment.append('Positive')

reviews_df['Star_Sentiment'] = star_sentiment

# Add clusters to dataframe
reviews_df['Cluster'] = clusters

In [284]:
reviews_df.head(15)

,Movie,Review,User_Rating,Star_Sentiment,Cluster
0,Moana,Moana is a return to the classic Disney formul...,9,Positive,1
1,Moana,"Wow, what a great great movie! The visuals in ...",9,Positive,3
2,Moana,Tell me if you've seen this movie:Princess wan...,5,Positive,1
3,Moana,Instant classic.Easily ranks among the best fr...,10,Positive,1
4,Moana,2016 has been a particularly strong year for a...,8,Positive,3
5,Moana,"From the opening lyrics to the final shot, Moa...",9,Positive,3
6,Moana,I have been largely unimpressed with Disney's ...,6,Positive,3
7,Moana,Henny's Movie Review: Moana (aka Vaiana)In thr...,8,Positive,3
8,Moana,"I saw ""Moana"" tonight in the local movie theat...",9,Positive,3
9,Moana,"So I watched Moana and boy, what a lazy movie ...",4,Negative,3


In [283]:
print('Total Positive reviews: ', len(reviews_df.loc[reviews_df['Star_Sentiment'] == 'Positive']))
print('Total Negative reviews: ', len(reviews_df.loc[reviews_df['Star_Sentiment'] == 'Negative']))

Total Positive reviews:  74
Total Negative reviews:  26


This dataframe provides us with the following information:

    - Movie: The name of the movie for which the review was written
    - Review: The text of the review submitted by the reviewer
    - User_Rating: The star rating attached to the review by the reviewer
    - Star_Sentiment: The sentiment assigned to the review based on the User_rating
        - >=5 is Positive
        - <5 is Negative
        - No rating is Positive
    - Cluster: The cluster assigned to each review based on our KMeans model

Using this dataframe, I next create the sentiment analyzer model using the Star_Sentiment column of as my baseline.

In [291]:
## In order to insure that I have Positive and Negative reviews in each dataset (train and test), 
## I need to specify classes on split
print(reviews_df.index[reviews_df['Star_Sentiment'] == 'Positive'])
print(reviews_df.index[reviews_df['Star_Sentiment'] == 'Negative'])

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 13, 16, 19, 27, 28, 30, 34,
            35, 36, 40, 41, 44, 45, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
            60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
            77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93,
            94, 95, 96, 97, 98, 99],
           dtype='int64')
Int64Index([ 9, 11, 12, 14, 15, 17, 18, 20, 21, 22, 23, 24, 25, 26, 29, 31, 32,
            33, 37, 38, 39, 42, 43, 46, 48, 49],
           dtype='int64')


In [302]:
# Split data into testing an training - 70/30 split
## Using 52 of the positive and 18 of the negative for train, and 22 of the positive and 8 of the negative for test
## This maintains the class percentage of the overall dataset for the training set (74% positive, 26% negative)
train_data = reviews_df.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                              21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,40,
                              41,44,45,47,50,51,52,53,54,55,56,57,58,59,60,61,62,
                             63,64,65,66,67,68,69,70,71,72,73,74,75,76,77]]
test_data = reviews_df.iloc[[37,38,39,42,43,46,48,49,78,79,80,81,82,83,84,85,86,87,88,89,90,
                       91,92,93,94,95,96,97,98,99]]

train_reviews = np.array(train_data['Review'])
train_sentiments = np.array(train_data['Star_Sentiment'])
test_reviews = np.array(test_data['Review'])
test_sentiments = np.array(test_data['Star_Sentiment'])

# Some sample reviews to test our model
sample_docs = [1, 6, 11, 16, 21]
sample_data = [(test_reviews[index], test_sentiments[index]) for index in sample_docs]

Load a sentiment vocabulary from the textbook and run as explained in the reference.

In [408]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    #review = normalize_accented_characters(review)
    #review = html_parser.unescape(review)
    #review = strip_html(review)
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative',
                                                                       'Neutral']], 
                                                              labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

In [431]:
def return_scores(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    #review = normalize_accented_characters(review)
    #review = html_parser.unescape(review)
    #review = strip_html(review)
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=['Predicted Sentiment', 'Polarity Score','Positive', 'Negative','Neutral'])
    return sentiment_frame

In [409]:
for review, review_sentiment in np.array(reviews_df[['Review', 'Star_Sentiment']]):
    print('Review: ')
    print(review)
    print('')
    print('Labeled Sentiment: ', review_sentiment)
    print('')
    final_sentiment = analyze_sentiment_vader_lexicon(review, threshold=0.1, verbose=True)
    print('-'*60)

Review: 
Moana is a return to the classic Disney formula, the clichés and characters ripped from a number of other animated films. However, the pure beauty and skill of the production rises the old story into new heights.Following from the success of Zootropolis, Moana follows a more traditional narrative we know and love; the princess who wishes for something more and is whisked on a supernatural adventure. We know this story so well yet Moana seems fresh and thrilling as if the plot was innovative. Perhaps this is due to the Polynesian setting or the morally ambiguous Maui, played perfectly by Dwayne Johnson, but most likely it is it the simple magic of Disney – the wonder for both children and adults has reached its peak with the perfection of the classic formula. For once, the clichés make the film more enjoyable. The quality of the animation helps too: it's clear they have reached the pinnacle of blending realistic textures with stylised designs, creating an aesthetic beauty that 

Review: 
Unlike "Bolt" and "Zootopia", Disney tried again another musical, i.e., people singing instead of talking. That apart, unlike other Disney musicals, e.g., "Mulan" and "Tangled", the songs in this film are very repetitive. I lost count of how many time the main character's song is reprised, with maybe slightly different lyrics every time, but I stopped caring after the second one. The explorer's song, one of the few with Polynesian language thrown-in and that doesn't sound like a cheap Western European cartoon musical, is somewhat "fine", although also repeated unnecessarily too many times. Maui's and Tamatoa's songs are happily jazzy and unfitting with the film, but helped me to survive till the obvious ending.Yes, the ending is squarely obvious after the first few minutes, just after the story of Maui's deed is told. The producers tried to make it less obvious by hiding the spiral on Te Ka's chest, but it's a Disney cartoon, who couldn't tell that Maui would come back to help

Labeled Sentiment:  Negative

     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            negative          -0.74     5.0%     8.0%   87.0%
------------------------------------------------------------
Review: 
Frozen...was a disappointment. :( and this is coming from a huge Disney geek. The reasons? 1. fails to explain how Elsa got those unique powers in the first place. 2. the parents death was a joke. so easy, and just put there as a necessary transition for future plot sake. 3. despite the fresh theme and sheer potential with snow and ice elements, there wasn't enough visual satisfaction. 4. it seemed like the movie worked with limited funding. the background scenes were so minimal! they could have done so much more with snow, the interior of the castle, the kingdom, the clothes, the mountain, etc 5. would have been better to see how Kristoff grew up with the trollsDisney movies are usually very complex, 


     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            positive           0.98    21.0%     0.0%   79.0%
------------------------------------------------------------
Review: 
Strong female leads are nowadays more desirable than they ever have been. Frozen capitalizes on this as strongly as possible, but fails at almost every turn. Their attempt at showing a more platonic love story is misguided and blunt with it's delivery. The story is full of plot holes and strange plot devices that make little to no sense within the context. The relationships feel forced and the message of the movie feels completely benign. Albeit, I am writing this from a 22 year old male's perspective, but having grown up with Disney movies and after the sheer gold that was Tangled, I have certain standards. Instead of going on a long winded explanation as to my complaints with this movie, I'm going to summarize in a numbered list


Labeled Sentiment:  Positive

     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            positive           0.84    47.0%     0.0%   53.0%
------------------------------------------------------------
Review: 
Pixar has done it AGAIN! 'Coco' is a yet another delightful ride from the prolific animation studio, who's winning streak seems to never end. And that's good for all, since 'Coco' is all heart & soul, rewarding the moviegoer wholeheartedly.'Coco' Synopsis: Aspiring musician Miguel, confronted with his family's ancestral ban on music, enters the Land of the Dead to find his great-great-grandfather, a legendary singer.'Coco' captures the themes of Life & Death, with genuine feeling. Themes of family, passion, defeat & redemption also are explored here, but with humor & real emotion. While I laughed for a good share of this winning story, I also found myself weeping in the film's final-act. This is a tru


Labeled Sentiment:  Positive

     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            positive           0.98    25.0%     8.0%   67.0%
------------------------------------------------------------
Review: 
This is truly one of the best Disney movies ever. I really enjoyed it when I first seen it, about when I was 6, and since then I watched it over and over again. I simply LOOOOVED the music. It's one of the best soundtracks I've ever heard. And, speaking of soundtrack, I just can't tell in words how much I love the song "Can You Feel The Love Tonight". Is one of my favorite songs ever. I truly believe that this motion picture could easily be nominated for The Best Picture, but The Best Song and the Best Original Score is really enough. I wonder if they will ever think to make a prequel of this movie in order to understand who is Mufasa, Scar, Sarabi and the other characters. In short, this movie will b

Labeled Sentiment:  Positive

     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            positive           0.95    23.0%     0.0%   77.0%
------------------------------------------------------------
Review: 
This is in my top 3 animation films of all time. I don't think that any animation film before it managed to enter the hearts of both children and adults at once like this one. An ensemble cast with such a gripping tale, great music and superb animation, made this one of the all time greats. The characters are so well flushed out and each scene is so memorable. I cannot stress just how good this film is. Without a doubt the peak of Disney and to this day they have not made anything this original and so influential. The creators gave us such a good villain and great hero's that in my opinion shows that an animated film can be just as good or if not better than a real life film.

Labeled Sentiment:  Posit

### <a name="Question-2"></a>Question-2

With the sentiment analysis in place, I now calculate the average, median, high, and low sentiment scores for each cluster.

In [504]:
def get_cluster_data(clustering_obj, review_data, feature_names, num_clusters, topn_features=10):
    # Create a dict to store the results
    cluster_details = {}
    
    # Get cluster centroids
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    
    # Get key features for each cluster
    # Get reviews belonging to each cluster
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index] for index in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features
        
        reviews = review_data[review_data['Cluster'] == cluster_num]['Review'].values.tolist()
        cluster_details[cluster_num]['Reviews'] = reviews
    
    return cluster_details

# To print the data from above in a clear format
def print_cluster_data(cluster_data):
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details: '.format(cluster_num))
        print('-'*20)
        print('Key features: ', cluster_details['key_features'])
        print('-'*40)

In [507]:
# Get cluster information
cluster_data = get_cluster_data(clustering_obj = km_obj, 
                                review_data=reviews_df, 
                                feature_names = feature_names, 
                                num_clusters=num_clusters,
                                topn_features = 5)

In [491]:
# Start by returning the polarity scores from the Analyzer in order to calculate by cluster
Polarity_score = []

for review in reviews_df['Review']:
    sentiment_frame = return_scores(review, threshold=0.1, verbose=True)
    Polarity_score.append(sentiment_frame['Polarity Score'].values[0])

In [494]:
# Add a column for Polarity Score
reviews_df['Polarity_Score'] = Polarity_score

In [495]:
reviews_df.head()

,Movie,Review,User_Rating,Star_Sentiment,Cluster,Polarity_Score
0,Moana,Moana is a return to the classic Disney formul...,9,Positive,1,1.00
1,Moana,"Wow, what a great great movie! The visuals in ...",9,Positive,3,1.00
2,Moana,Tell me if you've seen this movie:Princess wan...,5,Positive,1,0.92
3,Moana,Instant classic.Easily ranks among the best fr...,10,Positive,1,0.91
4,Moana,2016 has been a particularly strong year for a...,8,Positive,3,1.00


Above, we can see that we have the polarity score as well as the cluster for each review.

Using this, we can calculate the average, median, high, and low sentiment scores for each cluster.

In [502]:
print('Average: ')
print(reviews_df.groupby('Cluster', as_index=False)['Polarity_Score'].mean())
print('-'*40)
print('Median: ')
print(reviews_df.groupby('Cluster', as_index=False)['Polarity_Score'].median())
print('-'*40)
print('High: ')
print(reviews_df.groupby('Cluster', as_index=False)['Polarity_Score'].max())
print('-'*40)
print('Low: ')
print(reviews_df.groupby('Cluster', as_index=False)['Polarity_Score'].min())

Average: 
   Cluster  Polarity_Score
0        0        0.659000
1        1        0.937857
2        2        0.405238
3        3        0.770000
----------------------------------------
Median: 
   Cluster  Polarity_Score
0        0           0.980
1        1           0.965
2        2           0.990
3        3           0.980
----------------------------------------
High: 
   Cluster  Polarity_Score
0        0             1.0
1        1             1.0
2        2             1.0
3        3             1.0
----------------------------------------
Low: 
   Cluster  Polarity_Score
0        0           -0.95
1        1            0.60
2        2           -0.98
3        3           -1.00


In [509]:
# Key features of clusters from HW 7
print_cluster_data(cluster_data)

Cluster 0 details: 
--------------------
Key features:  ['movie', 'coco', 'best', 'simba', 'pixar']
----------------------------------------
Cluster 1 details: 
--------------------
Key features:  ['film', 'disney', 'animation', 'animated', 'films']
----------------------------------------
Cluster 2 details: 
--------------------
Key features:  ['elsa', 'anna', 'movie', 'disney', 'frozen']
----------------------------------------
Cluster 3 details: 
--------------------
Key features:  ['movie', 'moana', 'great', 'film', 'disney']
----------------------------------------


From the sentiment scores there are a number of inferences that can be made:

    - Average: Cluster 1 is made up primarily of positve reviews, cluster 2 is composed of mainly negative reviews, and clusters 0 and 3 contain a mixture of both with a slight leaning towards positive.
    
    - Median: For all clusters, there are a greater number of positive reviews than negative. This, however, could have been infered prior to sentiment analysis by observing that, overall, there are much more positive reviews in the dataset.
    
    - High: All clusters had a review that was mainly positive, this tells me that the clusters are not separated based on sentiment, but on other features. Like median, this information was already known from the key features the clusters were built around from HW 7. I could add polarity as a feature when building new clusters in the future as a way to see how they are affected by sentiment of the review.
    
    - Low: Clusters 0, 2, and 3 each have at least 1 intensely negative review. Cluster 1, however, continues to shine through as an overall positve cluster with the lowest polarity score being 0.60.